# Google Hash Code 2018 using ILP

## Import data

### import modules

In [1]:
import glob
import numpy as np
from gurobipy import *
from itertools import product

### get file list

In [2]:
files = glob.glob("data/*")
files = sorted(files)
files

['data/_a_test.in',
 'data/a_example.in',
 'data/b_should_be_easy.in',
 'data/c_no_hurry.in',
 'data/d_metropolis.in',
 'data/e_high_bonus.in',
 'data/test.in']

### select file to pick

In [3]:
FILE_N = 2

In [4]:
with open(files[FILE_N]) as file:
    file_iter = iter(list(file))
    
header = next(file_iter)
header_elem = iter(header.split(" "))

n_row = int(next(header_elem))
n_col = int(next(header_elem))
n_vehic = int(next(header_elem))
n_rides = int(next(header_elem))
bonus_point = int(next(header_elem))
n_turns = int(next(header_elem))

start_pos = np.zeros((n_rides,2), dtype=int)
end_pos = np.zeros((n_rides,2), dtype=int)
early_start_turn = np.zeros(n_rides, dtype=int)
late_end_turn = np.zeros(n_rides, dtype=int)

for i,e in enumerate(file_iter):
    elements = iter(e.split(" "))
    start_pos[i,0] = int(next(elements))
    start_pos[i,1] = int(next(elements))
    end_pos[i,0] = int(next(elements))
    end_pos[i,1] = int(next(elements))
    early_start_turn[i] = int(next(elements))
    late_end_turn[i] = int(next(elements))

## Preprocessing

### compute the length of rides

In [5]:
length_trip = np.sum(np.abs(start_pos - end_pos),axis=1)

### compute the latest a ride can start and the earliest a ride can finish

In [6]:
late_start_turn = late_end_turn - length_trip
early_end_turn = early_start_turn + length_trip
max_delay = late_start_turn - early_start_turn

### compute distance from ride to ride

In [7]:
start_distance = np.sum(start_pos,axis=1)
valid_transition = np.zeros((n_rides,n_rides),dtype=bool)
distance = np.zeros((n_rides,n_rides),dtype=int)

for i,j in product(range(n_rides),range(n_rides)):
    
    distance[i,j] = np.sum(np.abs(end_pos[i] - start_pos[j]))
    
    if early_end_turn[i] + distance[i,j] <= late_start_turn[j] and i != j:
        valid_transition[i,j] = True

## Create ILP model

In [8]:
m = Model()

Academic license - for non-commercial use only


### Initialize variables

In [9]:
start_transition = np.empty((n_rides,),dtype=np.object)
end_transition = np.empty((n_rides,),dtype=np.object)
delay = np.empty((n_rides,),dtype=np.object)
bonus = np.empty((n_rides,),dtype=np.object)

for i in range(n_rides):
    start_transition[i] = m.addVar(vtype=GRB.BINARY, name="t_S_"+str(i))
    end_transition[i] = m.addVar(vtype=GRB.BINARY, name="t_"+str(i)+"_E")
    delay[i] = m.addVar(lb=0,ub=max_delay[i],vtype=GRB.INTEGER, name="d_"+str(i))
    bonus[i] = m.addVar(vtype=GRB.BINARY, name="bonus_"+str(i))

transition = np.empty((n_rides,n_rides),dtype=np.object)

for i,j in product(range(n_rides),range(n_rides)):
    if valid_transition[i,j]:
        transition[i,j] = m.addVar(vtype=GRB.BINARY, name="t_"+str(i)+"_"+str(j))

### Add constrins

In [10]:
m.addConstr(sum(start_transition) <= n_vehic,"start_constrain")

for i in range(n_rides):
    m.addConstr(start_transition[i] + sum(transition[valid_transition[:,i],i]) == sum(transition[i,valid_transition[i,:]]) + end_transition[i], name="io_euqal_"+str(i))
    m.addConstr(start_transition[i] + sum(transition[valid_transition[:,i],i]) <= 1,name="at_most_one_"+str(i))
    m.addConstr(start_transition[i] + sum(transition[valid_transition[:,i],i]) >= bonus[i],name="bonus_if_ride_taken_"+str(i))
    
    m.addConstr(early_start_turn[i] + delay[i] + (1-start_transition[i])*(start_distance[i]-early_start_turn[i]) >= start_distance[i] , name="delay_S_"+str(i))
    
    m.addConstr((1 - bonus[i])*max_delay[i] >= delay[i],"bonus_apply_"+str(i))
    
for i,j in product(range(n_rides),range(n_rides)):
    if valid_transition[i,j]:
        m.addConstr(early_end_turn[i] + delay[i] + distance[i,j] <= early_start_turn[j] + delay[j] + (1-transition[i,j])*(max_delay[i]+distance[i,j]+early_end_turn[i]-early_start_turn[j]), name="delay_"+str(i)+"_"+str(j))


### Objective fuction

In [11]:
m.setObjective(bonus_point*sum(bonus) + 
               sum(length_trip[j]*transition[i,j] for i,j in product(range(n_rides),range(n_rides)) if valid_transition[i,j]) +
               sum(start_transition[i]*length_trip[i] for i in range(n_rides)),
               GRB.MAXIMIZE)

m.write("model.lp")
m.optimize()
m.write("model.sol")

Optimize a model with 44607 rows, 44306 columns and 304737 nonzeros
Variable types: 0 continuous, 44306 integer (44006 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [2e+01, 1e+03]
  Bounds range     [1e+00, 2e+03]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 38809 rows and 1538 columns
Presolve time: 0.85s
Presolved: 5798 rows, 42768 columns, 99777 nonzeros
Variable types: 0 continuous, 42768 integer (42486 binary)

Root relaxation: objective 1.769269e+05, 4698 iterations, 0.30 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 176902.000    0   26   -0.00000 176902.000      -     -    2s
H    0     0                    176119.00000 176902.000  0.44%     -    3s
H    0     0                    176535.00000 176902.000  0.21%     -    3s
H    0     0                    176877.0

### Parse solution

In [12]:
vfunc = np.vectorize(lambda v: np.int(v.x) if v is not None else 0)
start_transition = vfunc(start_transition)
transition = vfunc(transition)
bonus = vfunc(bonus)
delay = vfunc(delay)

In [13]:
vehicle_path = [[] for _ in range(n_vehic)]

for i,node in enumerate(np.argwhere(start_transition).flatten()):
    while node != -1:
        vehicle_path[i].append(node)
        nodes = np.argwhere(transition[node,:]).flatten()
        if len(nodes) < 1:
            node = -1
        else:
            node = nodes[0]

In [15]:
import os
out_dir = "out"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
out_file = os.path.join(out_dir,os.path.splitext(os.path.basename(files[FILE_N]))[0] + ".out")

with open(out_file,"w") as out:
    for vp in vehicle_path:
        out.write(str(len(vp)))
        for e in vp:
            out.write(" "+str(e))
        out.write("\n")